# FGGLIB Introduction

Factor graph grammars (FGG) are hyperedge replacement grammar that generate factor graphs.  <br>
They overcome the limited fixed structure of factor graphs, allow tractable inferences in many cases and can be useful to represent context free grammars - especially for probabilistic context free grammar which can generate more than one parse tree.  <br>
David Chiang et al. present factor graph grammars and algorithms for conjunction of and inference on them in their work Factor Graph Grammar.  <br>
As part of the course project of Advanced Formal Language Theory, we have implemented the algorithms in a library called FGGLIB. 
FGGLIB is a Python library structred in a same way as the paper and includes tests on a representative set of FGG examples.

### Installation

To install and use this library, download the source code and run _pip install -e ._ in the top directory.  <br>
In order to run all packages we recommend to use Python 3.10 and pip 21.


In [1]:
from IPython.display import Image
from fgglib.fg.factorgraph import Factorgraph
from fgglib.fg.hypergraph import Hypergraph
from fgglib.fg.vertex import Vertex
from fgglib.fg.edge import Edge
from fgglib.base.semiring import Real

## Hypergraph

First, we will start with building a basic hypergraph that consists of labeled vertices and edges. <br>
As an example we create a hyperedge that has six vertices and five edges as provided below. The elements of vertices and edges can be anything. 

 <img src="images/hypergraph.png" width="150"/>

In [2]:
hypergraph = Hypergraph()
vertex1 = Vertex('V1','v1')
vertex2 = Vertex('V2','v2')
vertex3 = Vertex('V3','v3')
vertex4 = Vertex('V4','v4')
vertex5 = Vertex('V5','v5')
vertex6 = Vertex('V6','v6')

edge1 = Edge(('V1, V2'),'e1')
edge2 = Edge({'V2, V3'},'e2')
edge3 = Edge({'V1, V3'},'e3')
edge4 = Edge({'V2, V3, V4, V5'},'e4')
edge5 = Edge({'V3', 'V5', 'V6'},'e5')

hypergraph.add_vertex(vertex1)
hypergraph.add_vertex(vertex2)
hypergraph.add_vertex(vertex3)
hypergraph.add_vertex(vertex4)
hypergraph.add_vertex(vertex5)
hypergraph.add_vertex(vertex6)

hypergraph.add_edge(edge1)
hypergraph.add_edge(edge2)
hypergraph.add_edge(edge3)
hypergraph.add_edge(edge4)
hypergraph.add_edge(edge5)

edge1.add_target(vertex1)
edge1.add_target(vertex2)
edge2.add_target(vertex2)
edge2.add_target(vertex3)
edge3.add_target(vertex1)
edge3.add_target(vertex3)
edge4.add_target(vertex2)
edge4.add_target(vertex3)
edge4.add_target(vertex4)
edge4.add_target(vertex5)
edge5.add_target(vertex3)
edge5.add_target(vertex5)
edge5.add_target(vertex6)

print(hypergraph.V)
print(hypergraph.E)

{Vertex: v5, Vertex: v4, Vertex: v6, Vertex: v1, Vertex: v2, Vertex: v3}
{{Edge e2: [Vertex: v2, Vertex: v3]}, {Edge e4: [Vertex: v2, Vertex: v3, Vertex: v4, Vertex: v5]}, {Edge e3: [Vertex: v1, Vertex: v3]}, {Edge e1: [Vertex: v1, Vertex: v2]}, {Edge e5: [Vertex: v3, Vertex: v5, Vertex: v6]}}


## Factor Graph

Factor graphs are probabilistic models which can be described as hypergraphs. They consist of variables, which are identical properties of the graph nodes and factors, a property of the hyperedges. These factors can be associated with some function and used to calculate a probability for a given variable assignment. <br>

Using FGGLIB an examplary factor graph from the original paper (Example3) can be generated, where vertices are drawn in circle and edges in rectangles connecting the vertices. <br>
 <img src="images/fg_exp3.png" width="500"/> <br>

In [5]:
from fgglib.fg.vertex import Vertex
from fgglib.fg.edge import Edge
from fgglib.autotesting.testenvironment import *
R=Boolean
fg = Factorgraph (R)
vertexSet = ('T0','T1','W2','T3','W4','T5','W6','T7')

fg.createVertices(None, vertexSet, R)

fg.createEdge('BOS','e1', {'T0'}, None, R)
fg.createEdge('T1|T0','e2', {'T0','T1'}, None, R)
fg.createEdge('W2|T1','e3', {'T1','W2'}, None, R)
fg.createEdge('T3|T1','e4', {'T1','T3'}, None, R)
fg.createEdge('W4|T3','e5', {'T3','W4'}, None, R)
fg.createEdge('T5|T3','e6', {'T3','T5'}, None, R)
fg.createEdge('W6|T5','e7', {'T5','W6'}, None, R)
fg.createEdge('T7|T5','e8', {'T5','T7'}, None, R)

fg.draw()




 <img src="graph.png" width="200"/> <br>

FGGLIB also enables usage of different semirings. As of now, semirings presented in the lecture are provided. <br>
For simplicity, the same notation was used.

ADD ASSIGNMENT

Factor Graph Grammars